In [5]:
import pandas as pd
import json
import sys
import yfinance
from googlemaps import Client as GoogleMaps
sys.path.append('../src')
sys.path.append('../src/data')
sys.path.append('..')
from api_config import GOOGLE_API_KEY
#from smartsheet_data import SAMPLE_SHEET_ID, PROJECT_SHEET_ID, SUPPLY_CHAIN_SHEET_ID, CFA_TRACKER_SHEET_ID, get_smartsheet_data, sheet_to_dataframe

In [4]:
import sys
import pandas as pd
sys.path.append('../data')
sys.path.append('..')
from data.projects import sync_projects
from app import db

In [31]:
from data.projects import clean_cfa_tracker_df, clean_sc_df, sheet_to_dataframe, sync_projects, get_smartsheet_data, SUPPLY_CHAIN_SHEET_ID, CFA_TRACKER_SHEET_ID

In [32]:
cfa_tracker_df_raw

NameError: name 'cfa_tracker_df_raw' is not defined

## STEP 1 - Clean Supply Chain DF

In [ ]:

# get columns
cols = ['Location_Number','Prototype', 'Region','Awarded_Wall_Panel_Supplier','SO_Amount','SO_Executed_Date','Panels_On_Site','sqft','Component_Model_Status','Wall_Panels','#_of_Ext_Wall_panel','#_of_Int_Wall_panel','sqft_of_Int_Wall_panel','sqft_of_Ext_Wall_panel']
# filter out any project that does not have a wall panels cost
sc_df=sc_df_raw[sc_df_raw['Wall_Panels'].notnull()][cols]
# drop extra NA values (usually at the end)
sc_df.dropna(how="all",inplace=True, axis=1)
sc_df['#_of_Ext_Wall_panel'] = sc_df[sc_df.apply(pd.to_numeric, errors='coerce').notna()].dropna()
# force location number to type (removes leading zeroes)
sc_df = sc_df.astype({'Location_Number':'int32'})
# fill empty numerical cells with zeroes
sc_df.fillna({'#_of_Int_Wall_panel':0,'#_of_Ext_Wall_panel':0,'sqft_of_Int_Wall_panel':0})
sc_df



,Location_Number,Prototype,Region,Awarded_Wall_Panel_Supplier,SO_Executed_Date,Panels_On_Site,sqft,Component_Model_Status,Wall_Panels,#_of_Ext_Wall_panel,#_of_Int_Wall_panel,sqft_of_Int_Wall_panel,sqft_of_Ext_Wall_panel
0,4055,P12 LS-L-R,Northeast Region,Mitsui,2018-06-18,2018-08-10,4880.0,#NO MATCH,91953.0,64.0,49.0,3199.0,6576.0
1,4066,P12 LS-L-R,Southwest Region,Mitsui,2018-08-09,2018-10-08,4880.0,#NO MATCH,96127.0,80.0,46.0,3013.0,7336.0
2,4022,P12 LS-M,Southwest Region,Mitsui,2018-10-01,2018-10-08,4880.0,#NO MATCH,90525.0,56.0,46.0,3157.0,5298.0
3,4048,P12 LC-L,West Region,Golden State,2019-01-04,2019-02-11,4844.0,#NO MATCH,80206.0,68.0,46.0,3401.0,5303.0
4,643,P12 SE-L,Southeast Region,Golden State,2019-02-22,2019-04-15,4859.0,#NO MATCH,91510.0,57.0,57.0,4109.0,5757.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,4435,P13 SE-L,Northeast Region,RedBuilt,2021-08-10,2021-11-04,5147.0,8. FABRICATED,127500.0,58.0,0.0,NaN,5618.0
85,832,P12 LS-L-R,Southeast Region,RedBuilt,2020-02-24,2021-11-08,4996.0,7. RELEASED TO PRODUCTION,73000.0,44.0,27.0,2299.0,5287.0
86,4765,P13 SE-M,Northeast Region,RedBuilt,2021-08-10,2021-12-02,4592.0,7. RELEASED TO PRODUCTION,126280.0,56.0,0.0,NaN,5364.0
99,4577,P13-LSR-L,Northeast Region,RedBuilt,2021-07-06,2022-04-28,4998.0,7. RELEASED TO PRODUCTION,148880.0,54.0,0.0,NaN,6179.0


## STEP 2 - Clean CFA DF

In [ ]:
cols = ["Location_Number","Address", "City", "State"]
cfa_tracker_df = cfa_tracker_df_raw[cols]
cfa_tracker_df["Location_Number"] = pd.to_numeric(cfa_tracker_df["Location_Number"],downcast='integer',errors='coerce').fillna(0)
cfa_tracker_df = cfa_tracker_df.astype({'Location_Number':'int32'})
cfa_tracker_df.dropna(axis=0,how="any",inplace=True)
cfa_tracker_df

C:\Users\jbuccola\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Location_Number,Address,City,State
0,563,1300 E Jackson Ave,Mcallen,TX
1,4055,12101 Winchester Rd,Lavale,MD
2,805,604 Holcomb Bridge Rd,Roswell,GA
3,4066,15500 Hwy 35,Buda,TX
4,4083,6885 Getwell Rd N,Southaven,MS
...,...,...,...,...
433,4342,3115 Chestnut St # 3133,Camp Hill,PA
434,4463,1600 W Armory Way,Seattle,OR
435,4594,Warwick Township,Warwick Township,PA
436,825,3066 Washington Rd,Augusta,GA


## STEP 3 - Merge Dataframes

In [ ]:
merged = sc_df.merge(cfa_tracker_df,how='left',left_on='Location_Number',right_on='Location_Number')
merged


,Location_Number,Prototype,Region,Awarded_Wall_Panel_Supplier,SO_Executed_Date,Panels_On_Site,sqft,Component_Model_Status,Wall_Panels,#_of_Ext_Wall_panel,#_of_Int_Wall_panel,sqft_of_Int_Wall_panel,sqft_of_Ext_Wall_panel,Address,City,State
0,4055,P12 LS-L-R,Northeast Region,Mitsui,2018-06-18,2018-08-10,4880.0,#NO MATCH,91953.0,64.0,49.0,3199.0,6576.0,12101 Winchester Rd,Lavale,MD
1,4066,P12 LS-L-R,Southwest Region,Mitsui,2018-08-09,2018-10-08,4880.0,#NO MATCH,96127.0,80.0,46.0,3013.0,7336.0,15500 Hwy 35,Buda,TX
2,4022,P12 LS-M,Southwest Region,Mitsui,2018-10-01,2018-10-08,4880.0,#NO MATCH,90525.0,56.0,46.0,3157.0,5298.0,3442 Palmer Hwy,Texas City,TX
3,4048,P12 LC-L,West Region,Golden State,2019-01-04,2019-02-11,4844.0,#NO MATCH,80206.0,68.0,46.0,3401.0,5303.0,2940 SW Cedar Hills Blvd,Beaverton,OR
4,643,P12 SE-L,Southeast Region,Golden State,2019-02-22,2019-04-15,4859.0,#NO MATCH,91510.0,57.0,57.0,4109.0,5757.0,352 Northside Dr E,Statesboro,GA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,4435,P13 SE-L,Northeast Region,RedBuilt,2021-08-10,2021-11-04,5147.0,8. FABRICATED,127500.0,58.0,0.0,NaN,5618.0,70 NJ Route 36,Eatontown,NJ
82,832,P12 LS-L-R,Southeast Region,RedBuilt,2020-02-24,2021-11-08,4996.0,7. RELEASED TO PRODUCTION,73000.0,44.0,27.0,2299.0,5287.0,211 Lakeshore Pkwy,Homewood,AL
83,4765,P13 SE-M,Northeast Region,RedBuilt,2021-08-10,2021-12-02,4592.0,7. RELEASED TO PRODUCTION,126280.0,56.0,0.0,NaN,5364.0,374 William S. Canning Boulevard,Fall River,MA
84,4577,P13-LSR-L,Northeast Region,RedBuilt,2021-07-06,2022-04-28,4998.0,7. RELEASED TO PRODUCTION,148880.0,54.0,0.0,NaN,6179.0,124 East Jericho Turnpike,Huntingtn Sta,NY


In [ ]:
cleaned_df = clean_projects_dataframe(combined_df)

In [ ]:
cfa_tracker_df_raw = sheet_to_dataframe(get_smartsheet_data(CFA_TRACKER_SHEET_ID)[1])

Starting ...
Loaded 439 rows from sheet: Panelization Project Tracking
Done


In [ ]:
cfa_clean_df = clean_cfa_tracker_df(cfa_tracker_df_raw)
cfa_clean_df

..\data\projects.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cfa_tracker_df["Location_Number"] = pd.to_numeric(cfa_tracker_df["Location_Number"],downcast='integer',errors='coerce').fillna(0)


,Location_Number,Address,City,State
0,563,1300 E Jackson Ave,Mcallen,TX
1,4055,12101 Winchester Rd,Lavale,MD
2,805,604 Holcomb Bridge Rd,Roswell,GA
3,4066,15500 Hwy 35,Buda,TX
4,4083,6885 Getwell Rd N,Southaven,MS
...,...,...,...,...
433,4342,3115 Chestnut St # 3133,Camp Hill,PA
434,4463,1600 W Armory Way,Seattle,OR
435,4594,Warwick Township,Warwick Township,PA
436,825,3066 Washington Rd,Augusta,GA


In [ ]:
cfa_clean_df[cfa_clean_df['Location_Number'] == 813]

,Location_Number,Address,City,State


In [ ]:
from data.commodities import get_historical_info

In [ ]:
lumber_df = get_historical_info('LBS=F')
lumber_df = lumber_df.drop(columns=["High","Low","Stock Splits", "Dividends","Volume"])
lumber_df = lumber_df.reindex(date_idx,method='ffill')
lumber_df = lumber_df.dropna(axis=0,how='any')
lumber_df


,Open,Close
date,,
2018-05-01,577.200012,591.799988
2018-05-02,595.099976,595.000000
2018-05-03,592.000000,583.500000
2018-05-04,595.500000,600.500000
2018-05-05,595.500000,600.500000
...,...,...
2021-10-28,621.700012,613.599976
2021-10-29,610.500000,591.599976
2021-10-30,610.500000,591.599976


In [ ]:
date_idx = pd.date_range(start='2018-05-01', end=datetime.datetime.today().strftime('%Y-%m-%d'),freq='D',name='date')
date_idx

DatetimeIndex(['2018-05-01', '2018-05-02', '2018-05-03', '2018-05-04',
               '2018-05-05', '2018-05-06', '2018-05-07', '2018-05-08',
               '2018-05-09', '2018-05-10',
               ...
               '2021-10-23', '2021-10-24', '2021-10-25', '2021-10-26',
               '2021-10-27', '2021-10-28', '2021-10-29', '2021-10-30',
               '2021-10-31', '2021-11-01'],
              dtype='datetime64[ns]', name='date', length=1281, freq='D')

In [ ]:
merged_df =  pd.merge(lumber_df,date_scaff,left_index=True, right_on='date',how='outer').reindex()
merged_df

,Open,Close,date
12,463.899994,466.399994,2017-11-01
13,468.600006,449.000000,2017-11-02
14,447.100006,449.000000,2017-11-03
17,448.600006,461.600006,2017-11-06
18,463.000000,464.799988,2017-11-07
...,...,...,...
1458,NaN,NaN,2021-10-17
1464,NaN,NaN,2021-10-23
1465,NaN,NaN,2021-10-24
1471,NaN,NaN,2021-10-30


In [ ]:
# Scaffolding for lumber prices
start_date = df['date'].sort_values(by='date',axis=1,ascending=True).iloc[0]


,date
0,2017-10-20
1,2017-10-21
2,2017-10-22
3,2017-10-23
4,2017-10-24
...,...
1469,2021-10-28
1470,2021-10-29
1471,2021-10-30
1472,2021-10-31


In [ ]:
import pandas as pd

In [ ]:
import datetime

In [6]:
from data.projects import sheet_to_dataframe, clean_cfa_cost_estimate_df, get_smartsheet_data, COST_ESTIMATE_SHEET_ID

In [8]:
cfa_cost_estimate_df_raw = sheet_to_dataframe(get_smartsheet_data(COST_ESTIMATE_SHEET_ID)[1])
cfa_cost_df = clean_cfa_cost_estimate_df(cfa_cost_estimate_df_raw)
cfa_cost_df

Starting ...
Loaded 118 rows from sheet: SC CFA Estimating Worksheet
Done


,CFA_Project_Number,Wood_Walls_CSW
0,0.0,46481.04
1,4055.0,91953.00
2,4066.0,96127.00
3,4022.0,90525.00
4,4048.0,80206.00
...,...,...
113,4765.0,126280.00
114,4435.0,127500.00
115,2188.0,91373.00
116,448.0,132405.00
